In [1]:
import cv2

# Load the cascade specifically trained for low resolution
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")

# To capture video from existing video.
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    _, img = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a slight blur to reduce noise
    gray = cv2.medianBlur(gray, 5)

    # Detect faces at a lower scale factor
    faces = face_cascade.detectMultiScale(gray, 1.05, 3)

    # Draw the rectangle around each face
    for x, y, w, h in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 1)

    # Display
    cv2.imshow("Face Window", img)

    #stop if length of video is reached
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        break

    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    

# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [2]:
import cv2
import face_recognition
import os


def load_known_faces(known_faces_directory):
    known_faces = []
    known_face_encodings = []

    for root, _, files in os.walk(known_faces_directory):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                full_path = os.path.join(root, filename)
                known_face = cv2.imread(full_path)
                bgr_to_rgb = cv2.cvtColor(known_face, cv2.COLOR_BGR2RGB)
                face_encoding = face_recognition.face_encodings(bgr_to_rgb)[0]

                known_faces.append(known_face)
                known_face_encodings.append(face_encoding)

    return known_faces, known_face_encodings

In [3]:
# Load the known faces and their encodings
known_faces_directory = r"M:/Database/karna/"
known_faces, known_face_encodings = load_known_faces(known_faces_directory)

In [4]:
def preprocess_video_frame(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply contrast enhancement using CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(2, (8, 8))
    gray = clahe.apply(gray)

    # Apply noise reduction using bilateral filtering
    gray = cv2.bilateralFilter(gray, 9, 75, 75)

    # Convert the grayscale image back to RGB format
    img_preprocessed = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    return img_preprocessed

In [5]:
import cv2
import face_recognition

# Load the cascade specifically trained for low resolution
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")

# Load the known faces and their encodings
known_faces, known_face_encodings = load_known_faces("known_faces")

# To capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    _, img = cap.read()

    # Preprocess the video frame to enhance face detection
    # img_preprocessed = preprocess_video_frame(img)
    img_preprocessed =img

    # Convert to grayscale
    gray = cv2.cvtColor(img_preprocessed, cv2.COLOR_BGR2GRAY)

    # Apply a slight blur to reduce noise
    gray = cv2.medianBlur(gray, 5)

    # Detect faces at a lower scale factor
    faces = face_cascade.detectMultiScale(gray, 1.05, 3)

    # For each face
    for x, y, w, h in faces:
        # Extract the face region
        face = img_preprocessed[y : y + h, x : x + w]

        # Convert the face region to RGB format
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Find the face encodings in the current frame
        face_encodings = face_recognition.face_encodings(face_rgb)

        # Check if there are any faces in the current frame
        if len(face_encodings) > 0:
            # Compare the face encodings in the current frame to the known faces
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encodings[0]
            )

            # If a match is found, identify the person
            if any(match for match in matches):
                index = matches.index(True)
                known_face = known_faces[index]

                # Draw a rectangle around the face and display the person's name
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(
                    img,
                    known_face,
                    (x, y - 20),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (255, 255, 255),
                    2,
                )

    # Display the frame
    cv2.imshow("Video", img)

    # Stop if the escape key is pressed
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [4]:
import cv2
import face_recognition
import os

# --- Configuration ---
known_faces_directory = r"M:/Database/karna/"
haar_cascade_file = "haarcascade_frontalface_alt2.xml"  

# --- Helper Functions ---
def load_known_faces(known_faces_directory):
    known_faces = []
    known_face_encodings = []

    for root, _, files in os.walk(known_faces_directory):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                full_path = os.path.join(root, filename)
                known_face = cv2.imread(full_path)
                bgr_to_rgb = cv2.cvtColor(known_face, cv2.COLOR_BGR2RGB)
                face_encoding = face_recognition.face_encodings(bgr_to_rgb)[0]

                known_faces.append(known_face)
                known_face_encodings.append(face_encoding)

    return known_faces, known_face_encodings


def preprocess_video_frame(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply contrast enhancement using CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(2, (8, 8))
    gray = clahe.apply(gray)

    # Apply noise reduction using bilateral filtering
    gray = cv2.bilateralFilter(gray, 9, 75, 75)

    # Convert the grayscale image back to RGB format
    img_preprocessed = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    return img_preprocessed


# --- Initialization ---
face_cascade = cv2.CascadeClassifier(haar_cascade_file)
known_faces, known_face_encodings = load_known_faces(known_faces_directory)
cap = cv2.VideoCapture(0)  

# --- Main Processing Loop ---
while True:
    _, img = cap.read()

    # Preprocess
    img_preprocessed = preprocess_video_frame(img) 

    # Grayscale + Blur for Haar Cascade
    gray = cv2.cvtColor(img_preprocessed, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5) 

    # Detect with Haar
    faces = face_cascade.detectMultiScale(gray, 1.05, 3)

    # Face Recognition for Each Detected Face
    for x, y, w, h in faces:
        face = img_preprocessed[y : y + h, x : x + w] 
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(face_rgb)

        if len(face_encodings) > 0:
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encodings[0]
            )

            if any(match for match in matches):
                index = matches.index(True)
                known_person_image = known_faces[index]

                # Draw rectangle in green (0, 255, 0)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2) 
        
                # Draw name below the face
                font = cv2.FONT_HERSHEY_DUPLEX
                name = os.path.basename(known_person_image)
                cv2.putText(img, name, (x, y + h + 20), font, 0.7, (255, 255, 255), 1)

    # Display output video 
    cv2.imshow("Facial Recognition", img)

    # Exit on 'ESC' key press
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

ValueError: too many values to unpack (expected 2)

: 